In [1]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
# 准备数据
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}
url = 'http://www.ceic.ac.cn/ajax/speedsearch?num=6&&page=1'

In [3]:
# 发送请求得到数据
def send_request(url):
    response = requests.get(url, headers=headers).content.decode()[1:-1]
    return json.loads(response)

In [4]:
# 查看数据
send_request()

TypeError: send_request() missing 1 required positional argument: 'url'

In [ ]:
#  分析数据 
def analysis_data(data):
    for i in data['shuju']:
        data_dataframe = pd.DataFrame(i, index=[i['LOCATION_C']])
        yield data_dataframe

In [ ]:
# 查看分析后的结果
# for i in analysis_data(send_request()):
#     print()

In [ ]:
# 写入文件
def save_svc(datas):
    for data in datas:
        if '地震数据3.csv' not in os.listdir(os.getcwd()):
            data.to_csv('地震数据3.csv', mode='a')
        else:
            data.to_csv('地震数据3.csv', mode='a', header=False)

In [ ]:
# save_svc(analysis_data(send_request()))

In [ ]:
# 批量获取文件
def get_all_data():
    url_list = []
    for i in range(1, 58):
        url = 'http://www.ceic.ac.cn/ajax/speedsearch?num=6&&page=%s' % i
        url_list.append(url)
    return url_list

In [ ]:
# get_all_data()

In [ ]:
def  run():
    # 生成地址列表
    url_list = get_all_data()
    # 便利地址发送请求
    for url in url_list:
        datas = send_request(url)
        # 清洗数据
        data_dataframe = analysis_data(datas)
        # 保存到文件
        save_svc(data_dataframe)

In [ ]:
run()

### 绘图

In [ ]:
# 近一年全球震级级别最高 top10  M
m_data = pd.read_csv('地震数据2.csv', usecols=['LOCATION_C', 'M'])

In [ ]:
m_data

In [ ]:
top10_data = m_data.sort_values(by='M', ascending=False).head(10)

In [ ]:
top10_data

In [ ]:
top10_data.plot(use_index=(top10_data['LOCATION_C'],top10_data['M'] ),kind='bar', figsize=(20,8), fontsize=20, grid='-', title='地震震级top10排名', rot=45)  

In [ ]:
top10_data.index = top10_data['LOCATION_C']

In [ ]:
top10_data

In [ ]:
top10_data.plot(kind='bar', figsize=(20,8), fontsize=20, grid='-', title='地震震级top10排名', legend=True) # use_index 

In [ ]:
top10_data.plot(kind='bar', figsize=(20,8), fontsize=20, grid='-', title='地震震级top10排名', legend=True, rot=45,) # use_index 

In [ ]:
# 近一年全球每月发生地震次数排名 O_TIME	LOCATION_C

In [ ]:
m_data = pd.read_csv('地震数据2.csv', usecols=['LOCATION_C', 'O_TIME'])

In [ ]:
m_data

In [ ]:
m_data['O_TIME'] = m_data['O_TIME'].map(lambda x: x.split('-')[1])

grouped_month = m_data.groupby('O_TIME') # 分组 

grouped_month_amount = grouped_month.O_TIME.count()#计数

top_month = grouped_month_amount.sort_values(ascending=False) # 排序

top_month

In [ ]:
top_month.values

In [ ]:
top_month.plot(kind='bar', figsize=(20, 8), label='排名')
plt.title('近一年全球每月发生地震次数排名')
plt.grid()
plt.xlabel('月份')
plt.ylabel('次数')
plt.xticks(rotation=45)
plt.legend()
for x, y in enumerate(list(top_month.values)):
    plt.text(x, y + 0.8, '%s' % round(y, 1), ha='center', bbox=dict(facecolor='red', alpha=0.5))


In [ ]:
# 近一年地震级别级数分布情况  M震级 
m_data = pd.read_csv('地震数据2.csv', usecols=['M'])

In [ ]:
grouped_month = m_data.groupby('M') # 分组 

grouped_month_amount = grouped_month.M.count()#计数

In [ ]:
len(grouped_month_amount)

In [ ]:
grouped_month_amount

In [ ]:
(max(grouped_month_amount.index) - min(grouped_month_amount.index)) / 0.5

In [ ]:
plt.figure(figsize=(20, 8), dpi=80)

plt.hist(x=grouped_month_amount.index, bins=int((max(grouped_month_amount.index) - min(grouped_month_amount.index))/1))
plt.xticks(grouped_month_amount.index, grouped_month_amount.index)